In [2]:
import torch
PATH_annotations = "../../../input/benetech-making-graphs-accessible/train/annotations/"
PATH_images = "../../../input/benetech-making-graphs-accessible/train/images/"
CHART_TYPE = {'vertical_bar':0,'horizontal_bar':1,'dot':2,'line':3,'scatter':4}
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE_ChartTypeClassify = 1
LEARNING_RATE_ChartTypeClassify = 0.001

import json, os
_,_,annotation_list = next(os.walk(PATH_annotations))
_,_,image_list = next(os.walk(PATH_images))
annotation_list.sort()
image_list.sort()
annotation_list_train = annotation_list[:55000]
annotation_list_test = annotation_list[55000:]
image_list_train = image_list[:55000]
image_list_test = image_list[55000:]

In [8]:
from PIL import Image
from shutil import copy

for idx in annotation_list:
    
    try:
    
        # read annotation message
        with open(PATH_annotations+idx) as file:
            this_annotation = json.loads(file.read())
        if this_annotation['chart-type']!='dot':
            continue
        # get image message
        img = Image.open(PATH_images+idx.split(".")[0]+".jpg")
        # get xlist and ylist
        x_list = [i['id'] for i in this_annotation['axes']['x-axis']['ticks']]
        y_list = [i['id'] for i in this_annotation['axes']['y-axis']['ticks']]

    # class, type, pos_of_whole_pic
        plot_bb = this_annotation['plot-bb']
        chart_x0 = plot_bb['x0']
        chart_y0 = plot_bb['y0']
        chart_x1 = plot_bb['x0']+plot_bb['width']
        chart_y1 = plot_bb['y0']+plot_bb['height']
        # main_chart = img.crop(box=(x0,y0,x1,y1))
        # main_chart.show()

    # tick_point
        for i in this_annotation['axes']['x-axis']['ticks']:
            idd = i['id']
            x_center = i['tick_pt']["x"]
            y_center = i['tick_pt']["y"]
            half_width = 8
            piece_x0 = x_center-half_width
            piece_x1 = x_center+half_width

    # crop_picture
            chart_peace = img.crop(box=(piece_x0,chart_y0,piece_x1,chart_y1))
            #chart_peace = chart_peace.resize((half_width*2, half_width*16))

    # get_label
            for textt in this_annotation['text']:
                if textt['id']==idd:
                    x_text = textt['text']
                    break
            for series in this_annotation['data-series']:
                if series['x']==x_text:
                    this_data = series['y']
                    break

        # save_picture
            name = f"{idx.split('.')[0]}_{idd}"
            if 1<=this_data<=12:
                chart_peace.save(f"./dataset/{this_data}/{name}.jpg")
            else:
                chart_peace.save(f"./dataset/13_and_more/{name}.jpg")
        
    except:
        print(idx)
    
 